<a href="https://colab.research.google.com/github/rbardon/atelier/blob/master/Atelier_Google_%C3%A0_compl%C3%A9ter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Développer pour l'IA : computer vision avec Transfer Learning

Atelier du 23 juillet 2019 Google Atelier Numérique

Animé par [Romain Bardon](https://www.linkedin.com/in/romain-bardon/) (IAdvance) et [Nicolas Lecointe](https://www.linkedin.com/in/nicolaslecointe/) (Macaron Software)


**Objectif**: suite à notre conférence "Intelligence Artificielle: savoir quand et comment l'utiliser pour vos projets" du 12 juillet à Google Atelier Numérique, nous voulons montrer ici les outils qui existent pour illustrer que la technologie du Deep Learning reste relativement accessible

Ce que l'on va utiliser : 
*   Librairies Tensorflow et Keras pour utiliser des réseaux de neurones.
*   Plusieurs jeux de données et modèles de réseaux de neurones existants.

# Fonctionnement du notebook

Un environnement d'execution dans une machine virtuelle / container, avec une interface Web pour interagir :

*   Presentation (MarkDown)
*   Code Source : ici Python
*   Terminal

cf. [Jupyter notebook](https://jupyter.org/) 
Essayer Jupyter sous différentes formes : [classique, jupyter lab, C++, R, Julia, Ruby, Scheme.](https://jupyter.org/try)

Les [commandes de bases](https://jupyter.org/documentation) (Documentation)

Dans le menu : Execution > Modifier le type d'execution


*   Type d'execution : Python 3
*   Accélérateur matériel : GPU

[GPU Tesla T4](https://www.nvidia.com/fr-fr/data-center/tesla-t4/)




In [0]:
!uname -a


In [0]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

# Quelques bases d'IA: la convolution et le transfer learning

## La convolution

Pour en apprendre plus sur l'utilisation classique des filtres : [https://perso.esiee.fr/~perretb/I5FM/TAI/convolution/index.html](https://perso.esiee.fr/~perretb/I5FM/TAI/convolution/index.html)

et de leur utilisation en Deep Learning : [A guide to convolution arithmetic for deep learning
](https://arxiv.org/abs/1603.07285)

![Convolution : exemple de calcul](https://perso.esiee.fr/~perretb/I5FM/TAI/_images/conv2.png)
![Un exemple de convolution](https://github.com/vdumoulin/conv_arithmetic/blob/master/gif/same_padding_no_strides.gif?raw=true)
source : [https://github.com/vdumoulin/conv_arithmetic](https://github.com/vdumoulin/conv_arithmetic)

"kernel/noyau", "step" ou "stride/decallage", "padding", "dilated (kernel)", "transposed" => upsample

Voici ce qu'un réseau de convolution apprend et "voit":

![Convolution : exemple de ce qu'il voit](https://blogs.nvidia.com/wp-content/uploads/2018/09/autos.png)

Les couches initiales du réseau apprennent à détecter les formes et contours simples. La complexité des éléments qu'apprend le réseau augmente au fur et à mesure que l'on avance dans les couches successives.

On peut donc en théorie (et en pratique!) utiliser un réseau ayant été entraîné sur un jeu de données et le réutiliser sur d'autres données pour répondre à une autre problématique. En effet, si les couches finales du réseau détectent des éléments très différents de notre problématique, on peut se servir d'une bonne partie du réseau qui lui détecte des formes plus ou moins génériques.

Ceci a deux avantages:
1. Partir d'un modèle déjà pré-entraîné avec de très nombreuses données et avec des capacités de calculs énormes (dont nous ne disponsons généralement pas). Ces modèles existent et ont généralement été entraînés sur des jeux de données de références (comme ImageNet)
2. Cette approche réduit considérablement le nombre d'image nécessaires pour notre nouveau problème, car il ne reste plus qu'à "spécialiser" le réseau à notre nouveau problème

## ResNet pour du transfer learning

[ResNet](https://arxiv.org/abs/1512.03385) est un réseau profond qui a gagné en 2015 le ILSVRC 2015 (ImageNet Large Scale Visual Recognition Challenge), utilisant un bloc "identité" pour réduire le problème du "vanishing gradient" (ie les valeurs des dérivés sont tellement petites que le réseau n'apprend plus):


<img src="https://neurohive.io/wp-content/uploads/2019/01/resnet-e1548261477164.png" width="400">

<img src="https://miro.medium.com/max/1400/1*aq0q7gCvuNUqnMHh4cpnIw.png" width="800">

<img src="https://miro.medium.com/max/1000/1*2ns4ota94je5gSVjrpFq3A.png" width="400">




# Etape 0 : import des différentes librairies

In [0]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import initializers

#CODE A COMPLETER - #importer la librairie Keras pour se servir du réseau ResNet. Documentation sur le site de Keras

# FIN


from tensorflow.python.keras.applications.resnet50 import preprocess_input
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import numpy as np

#import os
#os.environ['PYTHONHASHSEED']=str(1)


#tf.set_random_seed(1234)
#np.random.seed(1)

# Exercice 1 - Rural / Urbain

## Etape 1: importer les données

In [0]:
!git clone https://github.com/rbardon/atelier.git

In [0]:
%cd ./atelier/ex1 - urban & rural

!unzip rural_and_urban_photos

## Etape 2: visualiser les données

In [0]:
im_liste_rural = [1, 14, 26, 39]
im_liste_urban = [11, 24, 36, 43]


#pour voir les données de type "rural"
plt.figure(figsize=(18,18))
for i in range(0,4):
  plt.subplot(1,4,i+1)
  plt.imshow(mpimg.imread('./train/rural/rural' + str(im_liste_rural[i]) + '.jpeg'))
  plt.title('Rural')
  plt.axis('off')
  
#pour voir les données de type "urban"
plt.figure(figsize=(18,18))  
for i in range(0,4):
  plt.subplot(1,4,i+1)
  plt.imshow(mpimg.imread('./train/urban/urban_' + str(im_liste_urban[i]) + '.jpeg'))
  plt.title('Urban')
  plt.axis('off')

## Etape 3: un premier modèle avec convolution
Reseau de neurone convolutionnel ou convNet ou CNN

In [0]:
#les paramètres de base
image_size = 224
num_classes = 2
batch_size_ex1 = 24

In [0]:
#nous allons utiliser un générateur

data_generator_ex1b = ImageDataGenerator(rescale=1/255)

#CODE A COMPLETER
train_generator_ex1b = data_generator_ex1b.flow_from_directory(#code à compléter ici)

validation_generator_ex1b = data_generator_ex1b.flow_from_directory(#code à compléter ici)

#FIN

In [0]:
# un CNN assez simple
init_ex1b = initializers.RandomNormal(mean=0.0, stddev=0.05, seed=1)

model_ex1b = tf.keras.Sequential([
    layers.Conv2D(32,(3,3),activation='relu',input_shape=(224,224,3), kernel_initializer= init_ex1b),
    layers.MaxPool2D((2,2)),
    layers.Conv2D(64,(3,3),activation='relu', kernel_initializer= init_ex1b),
    layers.MaxPool2D((2,2)),
    layers.Conv2D(128,(3,3),activation='relu', kernel_initializer= init_ex1b),
    layers.MaxPool2D((2,2)),
    layers.Conv2D(128,(3,3),activation='relu',kernel_initializer= init_ex1b),
    layers.MaxPool2D((2,2)),
    layers.Flatten(),
    layers.Dense(512,activation='relu', kernel_initializer= init_ex1b),
    layers.Dense(num_classes, activation='softmax')   
])

#commande pour voir le détail de son modèle
model_ex1b.summary()

#avec Keras on doit compiler son modèle en spécifiant l'optimizer et la fonction coût
model_ex1b.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
# l'entraînement du modèle

#CODE A COMPLETER

history_ex1b = model_ex1b.fit_generator(#code à compléter ici)

#FIN

In [0]:
#on visualise la courbe d'apprentissage du modèle

plt.plot(history_ex1b.history['loss'])
plt.plot(history_ex1b.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train', 'cv'], loc='upper left')
plt.show()
plt.plot(history_ex1b.history['acc'])
plt.plot(history_ex1b.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['train', 'cv'], loc='upper left')
plt.show()

loss_ex1b_train, accuracy_ex1b_train = model_ex1b.evaluate_generator(train_generator_ex1b, verbose=0)
print('Accuracy sur training: %f' % (accuracy_ex1b_train))
print('Loss sur training: %f' % (loss_ex1b_train))

loss_ex1b_val, accuracy_ex1b_val = model_ex1b.evaluate_generator(validation_generator_ex1b, verbose=0)
print('Accuracy sur validation: %f' % (accuracy_ex1b_val))
print('Loss sur validation: %f' % (loss_ex1b_val))

## Etape 4: transfer learning

Les étapes à réaliser pour du transfer learning sont les suivantes:

1. Charger la base du modèle et ses poids, sans charger la "tête" du réseau

2. Définir la "tête" du modèle. A minima un classificateur avec le nombre de classes de notre problème

3. On fait tourner le modèle avec la base figée (i.e. trainable = false) pour que les paramètres de notre "tête" commencent à être affinés. Etape parfois facultative mais qui peut permettre d'éviter de diverger

4. On peaufine notre modèle si nécessaire en "relâchant" les dernières couches de notre base (i.e. trainable = true), afin de spécialiser notre réseau sur notre jeu de données. Un compromis à trouver entre précision et temps de calcul ("fine-tunning")

In [0]:
#nous allons utiliser un autre générateur

data_generator_ex1tl = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator_ex1tl = data_generator_ex1tl.flow_from_directory(
        'train',
        target_size=(image_size, image_size),
        batch_size=batch_size_ex1,
        class_mode='categorical',
        seed = 1)

validation_generator_ex1tl = data_generator_ex1tl.flow_from_directory(
        'val',
        target_size=(image_size, image_size),
        class_mode='categorical',
        seed = 1)

In [0]:
# creation d'un nouveau modèle vide. Ajout séquentiel de couches
model_ex1tl = Sequential()
#n'inclue pas la dernière couche du réseau. A priori on ne veut jamais enlever plus de couches
model_ex1tl.add(ResNet50(include_top=False, pooling='avg', weights='imagenet'))
model_ex1tl.add(Flatten())
# on ajoute un softmax (pourcentage de certitude sur des classes) pour avoir notre classificateur

# CODE A COMPLETER pour terminer le modèle: # on ajouter un layer de fin softmax

# FIN


# On spécifie que le ResNet est figé, il ne sera pas mis à jour à chaque itération (déjà entraîné)
model_ex1tl.layers[0].trainable = False

Type de calcul d'erreur

*   for binary_crossentropy: sigmoid activation, scalar target
*   for categorical_crossentropy: softmax activation, one-hot encoded target

Types de classification 

*   binary classification (two target classes)
*   multi-class classification (more than two exclusive targets)
*   multi-label classification (more than two non exclusive targets) 

In [0]:
#opt = SGD(lr=0.005, momentum=0.9)
model_ex1tl.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy']) # categorical_crossentropy  'sgd' ou 'Adam'

In [0]:
history_ex1tl = model_ex1tl.fit_generator(
        train_generator_ex1tl,
        epochs=10,
        steps_per_epoch=train_generator_ex1tl.n/batch_size_ex1,
        validation_data=validation_generator_ex1tl,
        validation_steps=1)

In [0]:
#on visualise la courbe d'apprentissage du modèle

plt.plot(history_ex1tl.history['loss'])
plt.plot(history_ex1tl.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train', 'cv'], loc='upper left')
plt.show()
plt.plot(history_ex1tl.history['acc'])
plt.plot(history_ex1tl.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['train', 'cv'], loc='upper left')
plt.show()

loss_ex1tl_train, accuracy_ex1tl_train = model_ex1tl.evaluate_generator(train_generator_ex1tl, verbose=0)
print('Accuracy sur training: %f' % (accuracy_ex1tl_train))
print('Loss sur training: %f' % (loss_ex1tl_train))

loss_ex1tl_train, accuracy_ex1tl_train = model_ex1tl.evaluate_generator(validation_generator_ex1tl, verbose=0)
print('Accuracy sur validation: %f' % (accuracy_ex1tl_train))
print('Loss sur validation: %f' % (loss_ex1tl_train))

# Exercice 2 - Cracks

## Etape 1: importer les données

In [0]:
#CODE A COMPLETER

#Changez le répertoire et dézippez le fichier pour télécharger les données

# FIN

## Etape 2: visualiser les données

In [0]:
im_liste_crack = [1, 408, 894, 1136]
im_liste_nocrack = [436, 725, 1124, 1238]

#CODE A COMPLETER

#Visualisez les données des listes crack et no crack définies ci-dessus

# FIN

## Etape 3: un premier modèle avec convolution
Reseau de neurone convolutionnel ou convNet ou CNN

In [0]:
image_size = 224
num_classes = 2 # "crack" ou "no crack" 
batch_size_ex2b = 24 # traitement des images par lot de 24 

In [0]:
# on utilise un générateur

data_generator_ex2b = ImageDataGenerator(rescale=1/255)

train_generator_ex2b = data_generator_ex2b.flow_from_directory(
        'train',
        target_size=(image_size, image_size),
        batch_size=batch_size_ex2b,
        class_mode='categorical')

validation_generator_ex2b = data_generator_ex2b.flow_from_directory(
        'val',
        target_size=(image_size, image_size),
        class_mode='categorical')

In [0]:
# un CNN assez simple

model_ex2b = tf.keras.Sequential([
    layers.Conv2D(32,(3,3),activation='relu',input_shape=(224,224,3)),
    layers.MaxPool2D((2,2)),
    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPool2D((2,2)),
    layers.Conv2D(128,(3,3),activation='relu'),
    layers.MaxPool2D((2,2)),
    layers.Conv2D(128,(3,3),activation='relu'),
    layers.MaxPool2D((2,2)),
    layers.Flatten(),
    layers.Dense(512,activation='relu'),
    layers.Dense(num_classes, activation='softmax')   
])

#commande pour voir le détail de son modèle
model_ex2b.summary()

#on compile le modèle
model_ex2b.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
# l'entraînement du modèle

history_ex2b = model_ex2b.fit_generator(
        train_generator_ex2b,
        epochs=10,
        steps_per_epoch=train_generator_ex2b.n/batch_size_ex2b,
        validation_data=validation_generator_ex2b,
        validation_steps=1)

In [0]:
#on visualise la courbe d'apprentissage du modèle

plt.plot(history_ex2b.history['loss'])
plt.plot(history_ex2b.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train', 'cv'], loc='upper left')
plt.show()
plt.plot(history_ex2b.history['acc'])
plt.plot(history_ex2b.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['train', 'cv'], loc='upper left')
plt.show()

loss_ex2b_train, accuracy_ex2b_train = model_ex2b.evaluate_generator(train_generator_ex2b, verbose=0)
print('Accuracy sur training: %f' % (accuracy_ex2b_train))
print('Loss sur training: %f' % (loss_ex2b_train))

loss_ex2b_val, accuracy_ex2b_val = model_ex2b.evaluate_generator(validation_generator_ex2b, verbose=0)
print('Accuracy sur validation: %f' % (accuracy_ex2b_val))
print('Loss sur validation: %f' % (loss_ex2b_val))

## Etape 4 : avec transfer learning

In [0]:
batch_size_ex2tl = 24

In [0]:
# on utilise un générateur
data_generator_ex2tl = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator_ex2tl = data_generator_ex2tl.flow_from_directory(
        'train',
        target_size=(image_size, image_size),
        batch_size=batch_size_ex2tl,
        class_mode='categorical')

validation_generator_ex2tl = data_generator_ex2tl.flow_from_directory(
        'val',
        target_size=(image_size, image_size),
        class_mode='categorical')

In [0]:
# creation d'un modèle
model_ex2tl = Sequential()

model_ex2tl.add(ResNet50(include_top=False, pooling='avg', weights='imagenet'))
model_ex2tl.add(Flatten())
model_ex2tl.add(Dense(num_classes, activation='softmax'))

# on fige le ResNet
model_ex2tl.layers[0].trainable = False

In [0]:
#on compile le modèle
model_ex2tl.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy']) # categorical_crossentropy  'sgd' ou 'Adam'

In [0]:
# l'entraînement du modèle
history_ex2tl = model_ex2tl.fit_generator(
        train_generator_ex2tl,
        epochs=6, 
        steps_per_epoch=train_generator_ex2tl.n/batch_size_ex2tl,
        validation_data=validation_generator_ex2tl,
        validation_steps=1)

In [0]:
#on visualise la courbe d'apprentissage du modèle

plt.plot(history_ex2tl.history['loss'])
plt.plot(history_ex2tl.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train', 'cv'], loc='upper left')
plt.show()
plt.plot(history_ex2tl.history['acc'])
plt.plot(history_ex2tl.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['train', 'cv'], loc='upper left')
plt.show()

loss_ex2tl_train, accuracy_ex2tl_train = model_ex2tl.evaluate_generator(train_generator_ex2tl, verbose=0)
print('Accuracy sur training: %f' % (accuracy_ex2tl_train))
print('Loss sur training: %f' % (loss_ex2tl_train))

loss_ex2tl_val, accuracy_ex2tl_val = model_ex2tl.evaluate_generator(validation_generator_ex2tl, verbose=0)
print('Accuracy sur validation: %f' % (accuracy_ex2tl_val))
print('Loss sur validation: %f' % (loss_ex2tl_val))

## Etape 5: amélioration du modèle

Créez un modèle en rajoutant des layers à la "tête" (dense) chacun suivi de dropout (voir doc).

Essayez de réduire la taille des batch

In [0]:
#CODE A COMPLETER

batch_size_ex2tl2 = #à compléter

#FIN

In [0]:
# on utilise un générateur
data_generator_ex2tl2 = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator_ex2tl2 = data_generator_ex2tl2.flow_from_directory(
        'train',
        target_size=(image_size, image_size),
        batch_size=batch_size_ex2tl2,
        class_mode='categorical')

validation_generator_ex2tl2 = data_generator_ex2tl2.flow_from_directory(
        'val',
        target_size=(image_size, image_size),
        class_mode='categorical')

In [0]:
# creation d'un nouveau modèle
model_ex2tl2 = Sequential()

model_ex2tl2.add(ResNet50(include_top=False, pooling='avg', weights='imagenet'))
model_ex2tl2.add(Flatten())

#CODE A COMPLETER

#Rajoutez 2 layers Dense, chacun suivi de dropout

#FIN

model_ex2tl2.add(Dense(num_classes, activation='softmax'))

# on fige le ResNet
model_ex2tl2.layers[0].trainable = False

In [0]:
# on compile le modèle
model_ex2tl2.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy']) # categorical_crossentropy  'sgd' ou 'Adam'

In [0]:
# l'entraînement du modèle
history_ex2tl2 = model_ex2tl2.fit_generator(
        train_generator_ex2tl2,
        epochs=6, 
        steps_per_epoch=train_generator_ex2tl2.n/batch_size_ex2tl2,
        validation_data=validation_generator_ex2tl2,
        validation_steps=1)

In [0]:
#on visualise la courbe d'apprentissage du modèle

plt.plot(history_ex2tl2.history['loss'])
plt.plot(history_ex2tl2.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train', 'cv'], loc='upper left')
plt.show()
plt.plot(history_ex2tl2.history['acc'])
plt.plot(history_ex2tl2.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['train', 'cv'], loc='upper left')
plt.show()

loss_ex2tl2_train, accuracy_ex2tl2_train = model_ex2tl2.evaluate_generator(train_generator_ex2tl2, verbose=0)
print('Accuracy sur training: %f' % (accuracy_ex2tl2_train))
print('Loss sur training: %f' % (loss_ex2tl2_train))

loss_ex2tl2_val, accuracy_ex2tl2_val = model_ex2tl2.evaluate_generator(validation_generator_ex2tl2, verbose=0)
print('Accuracy sur validation: %f' % (accuracy_ex2tl2_val))
print('Loss sur validation: %f' % (loss_ex2tl2_val))